In [181]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [182]:
data= pd.read_csv('mini.csv')

In [183]:
data['year'] = data['occurence'].astype(str).str[0:4]
data['year'] = data['year'].astype(int)

In [184]:
data['month'] = data['occurence'].astype(str).str[5:7]
data['month'] = data['month'].astype(int)

In [185]:
tmp=pd.DataFrame(data.groupby('customer_id')['cost'].mean())
tmp['trans_count']=data.groupby('customer_id')['cost'].count()
tmp['total']=data.groupby('customer_id')['cost'].sum()

In [186]:
data=data[data['year']==2018]

In [187]:
five_m=data[data['month']<=5]
target_month=data[data['month']==6]

In [188]:
data.head()

,customer_id,occurence,cost,item_id,year,month
1,13891,2018-02-10 17:35:11.000000,1090.0,828115,2018,2
4,58500,2018-03-09 20:57:29.000000,150.0,518554,2018,3
5,572181,2018-05-06 20:25:39.000000,990.0,656673,2018,5
6,616443,2018-07-09 16:48:20.000000,2390.0,678768,2018,7
7,114804,2018-05-06 13:39:20.000000,3990.0,958574,2018,5


In [189]:
five_m=five_m[['customer_id']].drop_duplicates(subset=['customer_id'], keep=False)

In [190]:
target_month=target_month[['customer_id']].drop_duplicates(subset=['customer_id'], keep=False)

In [191]:
target_month['target']='yes'
result=pd.merge(five_m,target_month,how='left',on=['customer_id'])
result['target']=(result.target.isnull()).astype('int')

In [192]:
total=pd.merge(result,tmp, how='inner', on=['customer_id'])
total.head()

,customer_id,target,cost,trans_count,total
0,568402,1,590.000000,1,590.0
1,530299,1,500.000000,1,500.0
2,112048,1,3858.333333,3,11575.0
3,561707,1,1346.666667,3,4040.0
4,560111,1,3890.000000,1,3890.0


In [193]:
y=total['target']
x=total.drop(['customer_id','target'],axis=1)
x.head()

,cost,trans_count,total
0,590.000000,1,590.0
1,500.000000,1,500.0
2,3858.333333,3,11575.0
3,1346.666667,3,4040.0
4,3890.000000,1,3890.0


In [194]:
x_train,x_test,y_train,y_test=train_test_split(x,y, train_size=0.8,random_state=17)


In [195]:
y_train.shape

(36847,)

In [196]:
x_train.shape

(36847, 3)

In [197]:
x_test.shape

(9212, 3)

In [198]:
tree = DecisionTreeClassifier(max_depth=3,random_state=17)
tree.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=17, splitter='best')

In [199]:
tree_predictions = tree.predict(x_test)


In [200]:
accuracy_score(y_test,tree_predictions)

0.9242292661745549

In [201]:
tree_params = {'max_depth': range(1,11)}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=17),tree_params,cv=5)                     

locally_best_tree.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=17,
                                              splitter='best'),
             iid='warn', n_jobs=None, param_grid={'max_depth': range(1, 11)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=N

In [202]:
print("Best params:", locally_best_tree.best_params_)
print("Best cross validaton score", locally_best_tree.best_score_)

Best params: {'max_depth': 1}
Best cross validaton score 0.9313105544549082


In [203]:
tuned_tree = DecisionTreeClassifier(max_depth=1,random_state=17)
tuned_tree.fit(x_test,y_test)
tuned_tree_predictions = tuned_tree.predict(x_test)
accuracy_score(y_test,tuned_tree_predictions)

0.9242292661745549

In [204]:
from ipywidgets import Image
from sklearn.tree import export_graphviz
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

dot_data= StringIO()
export_graphviz(tuned_tree, feature_names=[(i) for i in x.columns], out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01c\x00\x00\x00\xdc\x08\x06\x00\x00\x00\xa5)\xc0\xf…